In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [2]:
data_by_month=pd.read_excel("DatabyMonth-135-8years.xlsx")
index=pd.date_range(start='2011',end='2019',freq='M',name='Date')
data_by_month.index=index
data_by_month.head()

,Truck
Date,
2011-01-31,39
2011-02-28,41
2011-03-31,57
2011-04-30,61
2011-05-31,54


In [3]:
#data_by_month['month']=data_by_month.index.month
#data_by_month['quarter']=data_by_month.index.quarter

In [4]:
data_train=data_by_month.iloc[:80,0]
data_test=data_by_month.iloc[80:,0]

In [5]:
data_train.shape,data_test.shape

((80,), (16,))

In [6]:
data_train.head(20)

Date
2011-01-31    39
2011-02-28    41
2011-03-31    57
2011-04-30    61
2011-05-31    54
2011-06-30    71
2011-07-31    56
2011-08-31    39
2011-09-30    40
2011-10-31    48
2011-11-30    44
2011-12-31    40
2012-01-31    33
2012-02-29    40
2012-03-31    44
2012-04-30    35
2012-05-31    35
2012-06-30    31
2012-07-31    51
2012-08-31    39
Freq: M, Name: Truck, dtype: int64

In [7]:
X_train, y_train = [], []

for i in range(4, len(data_train)-4):
    X_train.append(data_train[i-4:i])
    y_train.append(data_train[i:i+4])

In [8]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [9]:
X_train.shape, y_train.shape

((72, 4), (72, 4))

In [10]:
x_scaler = MinMaxScaler()
X_train = x_scaler.fit_transform(X_train)

In [11]:
y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(y_train)

In [12]:
X_train.shape

(72, 4)

In [13]:
X_train = X_train.reshape(72, 4, 1)

In [14]:
reg = Sequential()
reg.add(LSTM(units = 200, activation = 'relu', input_shape=(4,1)))
reg.add(Dense(4))

In [15]:
reg.compile(loss='mse', optimizer='adam')

In [16]:
reg.fit(X_train, y_train, epochs = 100)

Epoch 1/100
3/3 [==============================] - 0s 6ms/step - loss: 0.2275
Epoch 2/100
3/3 [==============================] - 0s 7ms/step - loss: 0.2102
Epoch 3/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1940
Epoch 4/100
3/3 [==============================] - 0s 5ms/step - loss: 0.1782
Epoch 5/100
3/3 [==============================] - 0s 5ms/step - loss: 0.1609
Epoch 6/100
3/3 [==============================] - 0s 6ms/step - loss: 0.1418
Epoch 7/100
3/3 [==============================] - 0s 4ms/step - loss: 0.1182
Epoch 8/100
3/3 [==============================] - 0s 5ms/step - loss: 0.0928
Epoch 9/100
3/3 [==============================] - 0s 7ms/step - loss: 0.0650
Epoch 10/100
3/3 [==============================] - 0s 5ms/step - loss: 0.0428
Epoch 11/100
3/3 [==============================] - 0s 8ms/step - loss: 0.0354
Epoch 12/100
3/3 [==============================] - 0s 6ms/step - loss: 0.0344
Epoch 13/100
3/3 [==============================] - 0s 6ms/st

In [17]:
X_test, y_test = [], []

for i in range(4, len(data_test)-4):
    X_test.append(data_test[i-4:i])
    y_test.append(data_test[i:i+4])

In [18]:
X_test, y_test = np.array(X_test), np.array(y_test)

In [19]:
X_test

array([[20, 18, 27, 26],
       [18, 27, 26, 30],
       [27, 26, 30, 22],
       [26, 30, 22, 29],
       [30, 22, 29, 27],
       [22, 29, 27, 24],
       [29, 27, 24, 31],
       [27, 24, 31, 30]], dtype=int64)

In [20]:
X_test = x_scaler.transform(X_test)
y_test = y_scaler.transform(y_test)

In [21]:
X_test = X_test.reshape(8,4,1)

In [26]:
y_pred = reg.predict(X_test)

In [28]:
y_pred_inv = y_scaler.inverse_transform(y_pred)
y_test_inv = y_scaler.inverse_transform(y_test)

In [29]:
mean_absolute_error(y_test_inv,y_pred_inv)

4.920530676841736

In [24]:
def evaluate_model(y_test, y_predicted):
    scores = []
    
    #calculate scores for each day
    for i in range(y_test.shape[1]):
        mse = mean_squared_error(y_test[:, i], y_predicted[:, i])
        rmse = np.sqrt(mse)
        scores.append(rmse)
    
    #calculate score for whole prediction
    total_score = 0
    for row in range(y_test.shape[0]):
        for col in range(y_predicted.shape[1]):
            total_score = total_score + (y_test[row, col] - y_predicted[row, col])**2
    total_score = np.sqrt(total_score/(y_test.shape[0]*y_predicted.shape[1]))
    
    return total_score, scores

In [25]:
evaluate_model(y_test, y_pred)

(0.1329770457732831,
 [0.11379392522636342,
  0.10571867507214609,
  0.14279125174102858,
  0.16191584715263768])